In [ ]:
#Import Main Dependencies
import pandas as pd
from sqlalchemy import create_engine
from unicodedata import normalize

#Imported should you want to run any analyses
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Read the IMDB_data file
csv_file = "movie_metadata.csv"
IMDB_data_df = pd.read_csv(csv_file)
IMDB_data_df.head()

In [ ]:
# Drop the unnecessary columns
IMDB_data_df = IMDB_data_df.loc[:,['movie_title','actor_1_name','genres','budget','gross','country','title_year','imdb_score']]
IMDB_data_df.head()

In [ ]:
#Check for any null values
IMDB_data_df.isnull().any()

In [ ]:
#drop NA's in the Budget and Gross columns
IMDB_data_df.dropna(subset=['budget','gross'], inplace=True)

In [ ]:
#Rename the column to match with Streaming Dataframe columns
IMDB_data_df.rename(columns={"movie_title": "Title"},inplace = True)

#IMDB_data has Latin line breaks (special characters); Use normalize function to convert to regular form
IMDB_data_df['Title']=IMDB_data_df['Title'].str.normalize('NFKD')

#Strip any leading spaces in Title column
IMDB_data_df['Title']=IMDB_data_df["Title"].str.strip()

#Drop the duplicate data
IMDB_data_df.drop_duplicates()

In [ ]:
#Read in the Streaming dataset
csv_file = "MoviesOnStreamingPlatforms.csv"
streaming_data_df = pd.read_csv(csv_file)
streaming_data_df.head()

In [ ]:
#Drop the unncessary columns
streaming_data_df = streaming_data_df.loc[:,['Title','Rotten Tomatoes','Netflix','Hulu','Prime Video','Disney+']]
streaming_data_df.head()

In [ ]:
streaming_data_df.info()

streaming_data_df

In [ ]:
streaming_data_df.isnull().any()

In [ ]:
#Drop the NA's 
streaming_data_df.dropna(subset=['Rotten Tomatoes'], inplace=True)

#Drop the duplicates
streaming_data_df.drop_duplicates()

In [ ]:
#Convert fractions into decimals
streaming_data_df['Rotten Tomatoes'] = streaming_data_df['Rotten Tomatoes'].str.split("/").apply(lambda x: float(x[0]) / float(x[1]))
streaming_data_df.head()

In [ ]:
#Use InnerJoin on the datasets on the column 'Title' 
merge_df = pd.merge(IMDB_data_df, streaming_data_df, on="Title",how='inner')
merge_df

In [ ]:
merge_df1=merge_df.copy()

#Replace the values of the below 4 columns with its column name to identify the platform
for i in merge_df1[['Netflix','Hulu','Prime Video','Disney+']].columns:
    merge_df1[i].replace(1,i,inplace=True)
    merge_df1[i].replace(0,'',inplace=True)

#use function .apply() and .join() to join the 4 columns and assign to new column 'Streaming_Platform'
merge_df1['Streaming_Platform'] = merge_df1[['Netflix','Hulu','Prime Video','Disney+']].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
merge_df1

#use lstrip,rstrip,replace to remove the leading, trailing and extra comma's
merge_df1['Streaming_Platform']= merge_df1['Streaming_Platform'].map(lambda x: x.lstrip(',').rstrip(','))
merge_df1["Streaming_Platform"]=merge_df1["Streaming_Platform"].str.replace(",,,",",")
merge_df1["Streaming_Platform"]=merge_df1["Streaming_Platform"].str.replace(",,",",")

merge_df_final = merge_df1.loc[:,['Title','title_year','imdb_score','Rotten Tomatoes','actor_1_name','Streaming_Platform','budget','gross','country','genres']]

#Sort the final dataset in descending order on imdb_score and 'Rotten_Tomatoes'
merge_df_final=merge_df_final.sort_values(by = ['imdb_score','Rotten Tomatoes'], ascending = [False,False])

In [ ]:
merge_df_final

In [ ]:
#Create SQL Alchemy engine to postgres database "IMDB_db"
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'IMDB_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
#show the existing table names in the database
engine.table_names()

In [ ]:
#write the data in the dataframe IMDB_data_df to the database table "IMDB_data"
IMDB_data_df.to_sql(name='IMDB_data', con=engine, if_exists='replace', index=False)

In [ ]:
#write the data in the dataframe streaming_data_df to the database table "streaming_data
streaming_data_df.to_sql(name='streaming_data', con=engine, if_exists='replace', index=False)

In [ ]:
#Query the IMDB_data table to confirm the above data load
pd.read_sql_query('select * from public."IMDB_data"', con=engine).head()

In [ ]:
#Query the streaming_data table to confirm the above data load
pd.read_sql_query('select * from public."streaming_data"', con=engine).head()

In [ ]:
#write the data in the dataframe merge_df_final to the database table "IMDB_streaming_data"
merge_df_final.to_sql(name='IMDB_Streaming_data', con=engine, if_exists='replace', index=False)

In [ ]:
#Query the IMDB_streaming_data table to confirm the above data load
pd.read_sql_query('select * from public."IMDB_Streaming_data"', con=engine)